In [1]:
import pandas as pd
import numpy as np
import random
import itertools

### Load the student dataset

In [2]:
path = 'data/student_data/final_student_merged_datasetA.csv'
df = pd.read_csv(path, index_col = False)

In [3]:
ug_df = df.iloc[:4746,].reset_index(drop=True)
grad_df = df.loc[4746:,].reset_index(drop=True)

In [4]:
def generate_pairings(df, overlap_ratio):
    n = int(df.shape[0]*overlap_ratio) #Number of students we wish to match
    m = 3 #Number of members in a team
    b = []
    temp = []
    final_teams = []
    teams = []
    flag = 0
    for i in range(1,n+1):
        b.append(i)
    temp = b.copy()
    while True:
        if(len(temp)<m and flag==m):
            if teams not in final_teams:
                final_teams.append(teams)
            if len(temp) != 0:
                final_teams.append(temp)
            break
        if(flag<=m-1):
            randp = random.randint(1,n)
            if(randp in temp):
                temp.remove(randp)
                teams.append(randp)
                flag += 1
                continue
            elif(randp not in temp):
                continue
        elif(flag == m):
            final_teams.append(teams)
            flag = 0
            teams = []
            continue
        else:
            print("Error")
            break
    return final_teams

In [5]:
def get_overlap_ratio(pref_dict, overlap_ratio):    
    matched_set = set()
    for k,v in pref_dict.items():
        for room_pref in v:
            if k in pref_dict[room_pref]:
                matched_set.update([k,room_pref])
    return len(matched_set)/len(pref_dict)

In [6]:
def roommate_preferences(data, overlap_ratio):
    df = data.copy(deep = True)
    df['roommate_preferences'] = np.empty((len(df), 0)).tolist()
    final_pairings = generate_pairings(df,overlap_ratio)
    for pairing in final_pairings:
        try:
            a, b, c = pairing[0], pairing[1], pairing[2]
            a_ruid , b_ruid, c_ruid = df.loc[a,"RUID"], df.loc[b,"RUID"], df.loc[c,"RUID"]
            df.loc[a,'roommate_preferences'].extend([b_ruid,c_ruid])
            df.loc[b,'roommate_preferences'].extend([a_ruid, c_ruid])
            df.loc[c,'roommate_preferences'].extend([b_ruid])
        except:
            pass
    print(f"Overlap Ratio obtained is {get_overlap_ratio(dict(zip(df.RUID, df.roommate_preferences)),overlap_ratio)} vs the desired {overlap_ratio}")
    return df

In [7]:
ug_df_with_pref = roommate_preferences(ug_df, 0.75)
grad_df_with_pref = roommate_preferences(grad_df, 0.75)

Overlap Ratio obtained is 0.7496839443742098 vs the desired 0.75
Overlap Ratio obtained is 0.75 vs the desired 0.75


In [8]:
final_student_merged_dataset_A = pd.concat([ug_df_with_pref,grad_df_with_pref], ignore_index=True)

In [11]:
final_student_merged_dataset_A.to_csv('data/student_data/datasetA_with_roommate_pref.csv', index = False)